# Supplementary Labsheet 0: Measuring Battery Voltage

This suppplementary labshoot investigates measuring the battery voltage of your robot whilst it is operating.  This labsheet is quite straight-forward and will introduce you to reading analog voltages.  Because this is a supplementary labsheet, you are expected to use your own initiative and critical thinking to solve problems and conduct investigations.

Measuing the battery voltage of the robot might be useful to:
- detect and alert you to a critically low battery (which would alter the performance of your robot)
- to adjust system parameters depending on your measured battery voltage.
- to develop autonomous optimisations of robot behaviour (e.g. optimised usage of battery whilst moving).
- an alternative proprioceptive sensor input relating the robot's interaction with the external environment.
- You can review guidance from the manufacturer Pololu on how the batteries may effect the motor performance <a href="https://www.pololu.com/docs/0J83/5.3">here</a>.

In this labsheet, we investigate some electronic schematics to understand how we can measure the battery voltage.



<br><br><br><br>

# A Look at Schematics

Pololu offer the full schematic for the 3Pi+ for free from their website, <a href="https://www.pololu.com/file/0J1790/3pi-plus-32u4-control-board-schematic.pdf">here</a>.  We will look at the relevant aspects for measuring the battery voltage.


A general description of the power sub-system for the 3Pi+ is provided in the documentation, <a href="https://www.pololu.com/docs/0J83/5.7">here</a>.  This description tells us:
- **VBAT**: is a pin connected to the batteries, which may be between 4.8volts to 6volts, depending on the type of AA-cell batteries installed.
- **VSW**: is the battery voltage (VBAT), after it has been through a power switching circuit (on/off) and a reverse-protection circuit (to attempt to prevent damage if the batteries are installed incorrectly).
- **VM**: is the regulated voltage supplied to the motors at 8v, which occurs after VSW.
- That VSW can be monitored through the Arduino pin **A1**.  

From the lectures, you should remember that an <a href="https://en.wikipedia.org/wiki/Analog-to-digital_converter">Analog-to-Digital Converter (ADC)</a> makes a comparison between a voltage present at a pin and a voltage reference.  This then presents two interesting questions concerning measuring our robot battery voltage:


1. If the robot batteries can run out (VSW voltage will drop over time), how can the robot make a comparison to a voltage reference on the same circuit?  (e.g., won't the analog-to-digital converter reference voltage change as the batteries change?)

2. If the robot microcontroller operates on 5volts, how can it make a comparative measurement to a larger battery voltage (e.g. 6 volts)?


When we look through schematics, any labels made to pins will be consistent throughout the document.  Therefore, if you see a label such as **VCCREG** on one page, this will be electronically connected to another reference of **VCCREG** on another page.  We can therefore follow the labels through the circuitry to gain an understanding of the operation of the circuit.




# Voltage Reference

When working with microcontrollers it is possible to provide an external voltage as a reference for the ADC as an alternative to the "internal" system voltage of 5v (in this case).

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/3PI_aref.png?raw=true"/>
</p>

In the above schematic, this large yellow block is in fact our 32u4 microcontroller which we have been using.  You can observe in small blue writing the label for the yellow block `ATmega32U4`.  We can see the many pins available of the microcontroller.  We can use the labels in red writing to work out through the rest of the schematic what connections are made across different devices.  

We can also observe some standard connections we would expect to see in other microcontroller systems.  These are a connection a voltage supply for logic (here, 5 volts labelled `5V`, going into VCC on the yellow block), and and `0v` or `ground`, which is indicated by the 3 horizontal lines in a triangle shape.  

Because the 5volts supplied to the 32u4 is going to be used to determine all computational logic (e.g., is this high or low, 0 or 1?), the 5volt supply is regulated to be as consistent as possible.  If the 5volts were unstable, our logic and therefore computation might become unreliable.  This means that the logic supply voltage 5v it is not a direct connection from our batteries.    As a matter of interest, let's investigate how the voltage is supplied to the 32u4:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/3PI_power_reg.png?raw=true"/>
</p>

In the above circuit we can see two yellow blocks.  These have been taken from page 4 of the schematics.  These are two more integrated circuits (electronic circuits made on a wafer of silicon, usually encapsulated in black plastic), which look like black squares with tiny writing on them on the real circuit board of your 3Pi+.  

The top yellow block in the circuit diagram above is not labelled with a part number - which is a little unusual.  We can probably find the chip and it's part number on the real circuit board with a little investigation.  This block is performing power regulation to 5volts.  This means that, as the input voltage `VSW` varies, this chip will attempt to produce a stable 5V output.  This chip, is likely to be a high-frequency <a href="https://en.wikipedia.org/wiki/Switched-mode_power_supply">switch-mode converter</a>.  

A switch-mode converter is an amazing innovation that allows for the regulation of voltage at extremely high efficiency - up to 95% efficiency in some cases (e.g., only 5% of energy is lost).  Prior to switch-mode, linear converters would step-down or step-up voltage (shift the voltage level) but usually produce a lot of heat as wasted energy. To make this worse, you'd often need a fan to keep the electronics cool (using more energy, shortening battery life!).  For any type of voltage transformer, we can apply a general understanding that voltage and current can be exchanged, e.g., less current for more voltage, and vice-versa.

We can observe that `VSW` is provided as in input to the block (on the left hand side, into the pin labelled 6, VBAT).  `VSW` is the supply from the battery after the power (on/off) switch. Presumably the label `VSW` is **V**oltage after **Sw**itch.  `R5V` is produced on the right hand side, out of the pin labelled 2, VOUT.  Here presumably the label is **R**egulated **5V**.  However, `R5V` is not fed directly to our 32u4 microcontroller.  

In the bottom yellow block, labelled in blue `TPS2113A`, will select either the 5volts provided from a USB cable connection (when you have the 3Pi+ plugged into your computer), or the `R5V` supply previously discussed.  This allows the 32u4 to be powered from either source, and for you to plug-and-unplug the cable.  

From this second yellow block, we see on the right the output `5V`, which we can trace back to the connections we saw to the ATmega32U4 block in the schematic.  More information on this regulation process can be found <a href="https://www.pololu.com/docs/0J83/all#5.7"> in the documentation</a>.

Because we are interested in measuring the battery voltage, it is interesting to locate the analog reference (**AREF**) on the schematic.  When we instruct the 32u4 to make an `analogRead()`, it must make a measurement relative to a known voltage reference source.  

The `AREF` is a pin on the 32u4 (the first large yellow block we reviewed.  We can see that AREF is connected through a capacitor to ground (0 volts).  This tells us that the analog reference pin is not connected to an external reference source by default.  

In other circumstances, it might be useful to supply a different reference voltage to the ADC, so that the resolution of the ADC maps (discretises) across a different voltage range.  For instance, perhaps it would be useful to have the 10-bit ADC representation across just 1volt range.

Whilst this `AREF` pin exists, it is normal for the `AREF` source to be configured in software to work from the internal voltage reference source.  Therefore, we can expect AREF to be the 5V supplied to the 3Pi+ microcontroller, labelled as **VCC** receiving **5V**.  On some microcontroller systems, this might be 3.3volts, or another voltage - we should try not to make any assumptions.

Having followed this circuit schematic through, we can summarise that the `AREF` for the ADC is taking an internal regulated 5volts as a reference source.  This means that, even as the battery voltage changes, a measurement can be taken against a consistent reference source of 5volts.  



# Measuring a larger voltage:

From our previous discussion, we know that our `analogRead()` will make a measurement of voltage made against a 5volt reference.  However, we can expect four AA cell batteries to have a larger voltage of something like 6volts.  

If we search the schematic for the label **A1** (where we will measure the battery voltage **VSW**), we can find the following circuit diagram:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/batt_pd.png?raw=true"/>
</p>

You may recognise the above configuration of resistors as a <a href="https://learn.sparkfun.com/tutorials/voltage-dividers/all">potential divider</a>.  

From our previous discussion, following the path of voltage regulation, we also know that VSW is prior to any regulation.  Therefore, we can expect to observe in our measurements of the pin `A1` any induced noise or ripple to the power supplied from the batteries.  

From the above circuit diagram, we can read that the voltage present at `VSW` is divided by the 20k and 10k resistors in series, with A1 reading from the junction (labelled as **BATLEV**).  

The choice of 20k and 10k resistors makes the potential divider quite convenient - it essentially divides the voltage at VSW into thirds, presenting 1/3 of the level of VSW to A1.  

This is a simple way to bring a larger voltage down to be measured.  However, it should be noted that it is not necessarily safe.  A particularly large voltage presented at VSW could be divided down to produce a voltage larger than 5V for our 32u4 microcontroller - potentially causing damage.  With this resistor configuration, up to 15volts could be divided down to a safe value for measurement (one third of 15v is 5v).  

This "dividing down to 1/3 of VSW" will also need to be remembered when we want to calculate the voltage measured at the pin A1.  With four AA cell batteries, we can expect `VSW` to be a voltage around 6volts.  Therefore, if our four AA cells had an ideal voltage of 6 volts, we would measure this as 2volts (one third) during an `analogRead()`.  

To understand why a third of VSW is presented at A1, we can look at the equation for a potential divider:

<p align="center">
<img src="https://cdn.sparkfun.com/assets/e/7/6/3/c/511968d9ce395f7c54000000.png"/>
</p>

If we substitute our circuit labels into the equation above, we achieve:

$$M_{adc} = VSW \times \frac{R5}{R4+R5}$$

Where, $M_{adc}$ is the digital measurement taken by our ADC through `analogRead(A1)`, `VSW` is the unknown battery voltage (which we wish to discover), and `R5`, `R4` are the resistors in the above schematic labelled in small blue writing.

We can then substitute our known values:

$$M_{adc} = VSW \times \frac{10000}{20000+10000}$$

Which simplifies to:

$$M_{adc} = VSW \times 0.33$$

Here, it is important to note that $M_{adc}$ is not a voltage, but a binary representation of how the voltage at pin `A1` corresponds to the reference source `AREF`.  To proceed, we must convert the digital measurment $M_{adc}$ to volts.



<br><br><br><br>

<br><br><br><br>

## Exercise 1

From the <a href="http://ww1.microchip.com/downloads/en/DeviceDoc/Atmel-7766-8-bit-AVR-ATmega16U4-32U4_Summary.pdf">datasheet</a> for the 32u4 microcontroller, we know that the ADC has a 10-bit representation of a continuous analog voltage.  Therefore, when we receive a number from the analogRead(), it is not in the units of volts.   Our measurement $M_{adc}$ is a binary representation of how the voltage at pin `A1` corresponds to the reference source `AREF`.

We know that the `AREF` is using a 5volt reference for the ADC unit by default. It is possible to convert that 10-bit value into units of voltage ($V_{adc}$):


$$V_{adc} = M_{adc} \times \frac{AREF}{2^{10}}$$

In the above
- `AREF` is the total voltage used as a reference for `analogRead()`.  
- $2^{10}$ is the 10-bit representation.  Therefore, we use $AREF/2^{10}$ to equate a voltage represented by 1-bit.  

    
$$V_{adc} = M_{adc} \times \frac{5volts}{2^{10}} $$

However, remember that the voltage measured at pin `A1` is 1/3 of VSW.  Therefore:

$$VSW \times 0.33 = V_{adc}$$

$$VSW = \frac{V_{adc}}{0.33}$$

1. Write a simple program for your robot to measure your battery voltage within `loop().`
    - Be sure to use the appropriate variable type.
    - Remember that we are measuring the battery voltage, and this means you must activate battery power (the on/off button, and blue LED).
    - View your result using the Serial Monitor and/or Serial Plotter.
    - Check that the reported voltage appears reasonable (what would you expect?).


2. Write a simple Arduino sketch to periodically activate the motors at high speed, and then to zero speed.  For example, 2 seconds on, and 2 seconds off.  
  - What do you observe on the Serial Plotter?
  - Experiment with different motor speeds and observe the results on your plotter.
  - Experiment with your robot on different surfaces and observe the results on your plotter.
  - Experiment with your robot on different inclination of surfaces (ramps).
  - Experiment with your robot running into an obstruction.
  - Write some code to filter your measurement of the battery voltage to reduce the measurement noise.  

3. It is possible to set a different internal `AREF` voltage.  The <a href="https://www.arduino.cc/reference/en/language/functions/analog-io/analogreference/">Arduino Documentation</a> states that, for devices using the  32u4 microcontroller:
  - `analogReference( DEFAULT );` utilises a 5volt reference.
  - `analogReference( INTERNAL );` utilises a 2.56volt reference.
  - **Validate:** read the documentation to utilise `analogReference( INTERNAL );`, and adjust your calculation of `VSW` to use 2.56 volts.
  - **Hypothesise:** What benefit might there be to setting `AREF` to 2.56volts?
  - **Hypothesise:** What tasks could benefit from measuring the battery voltage, and could this be used to prove the utility of changing `AREF`?







